In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
len(dataset[0])

9

In [8]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [9]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31)
y test shape: (490, 31, 2)


In [10]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=510)

In [11]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=240, sampling_rate=1,
                               batch_size=250)

In [12]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [13]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [14]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [15]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 240, 31)
y test shape: (250, 31, 2)


In [16]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
# regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
regressor.add(LSTM(units=10, input_shape=(240, 31), dropout=0.1))

# The output layer
# regressor.add(LSTM(25))
regressor.add(Dense(62, activation='relu'))
regressor.add(Reshape((31,2)))
# regressor.add(softmax(axis = -1))
regressor.add(Lambda(lambda  x: softmax(x, axis = -1)))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# regressor.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=100,batch_size=10, validation_split=0.1, callbacks = [EarlyStopping(monitor='val_acc', patience=50),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

Train on 459 samples, validate on 51 samples
Epoch 1/100
459/459 [==============================] - 10s 22ms/step - loss: 0.6931 - acc: 0.5066 - val_loss: 0.6927 - val_acc: 0.5266
Epoch 2/100
459/459 [==============================] - 10s 23ms/step - loss: 0.6927 - acc: 0.5127 - val_loss: 0.6927 - val_acc: 0.5092
Epoch 3/100
459/459 [==============================] - 10s 22ms/step - loss: 0.6924 - acc: 0.5208 - val_loss: 0.6926 - val_acc: 0.5127
Epoch 4/100
459/459 [==============================] - 10s 22ms/step - loss: 0.6921 - acc: 0.5212 - val_loss: 0.6925 - val_acc: 0.5089
Epoch 5/100
459/459 [==============================] - 10s 23ms/step - loss: 0.6917 - acc: 0.5206 - val_loss: 0.6927 - val_acc: 0.5076
Epoch 6/100
459/459 [==============================] - 10s 21ms/step - loss: 0.6915 - acc: 0.5219 - val_loss: 0.6927 - val_acc: 0.5104
Epoch 7/100
459/459 [==============================] - 10s 21ms/step - loss: 0.6914 - acc: 0.5219 - val_loss: 0.6927 - val_acc: 0.5085
Epoch 8/10

In [17]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [18]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [19]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

0.5070967741935484

In [20]:
model_name = '../model/LSTM/my_model3.h5'
regressor.save(model_name)  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [21]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model(model_name, custom_objects={"softmax": softmax})

In [22]:
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int
(sum(y_test[:, :, 1] == label_load[:, :, 1])/(y_test.size/2)).sum()

0.5070967741935484